# Propagation of polarized light through a vector-Apodizing Phase Plate 

This tutorial introduces the implementation of polarized light in HCIpy and the correct polarization implementation of a vector apodizing phase plate.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import matplotlib as mpl
#mpl.rc('image', interpolation='nearest', origin='lower')
#mpl.rcParams.update({'font.size': 32})
#plt.rc('figure', figsize=(10.0, 10.0))
from matplotlib import animation, rc
from IPython.display import HTML


from hcipy import *

In [ ]:
pupil_grid = make_pupil_grid(512)

vAPP_phase = Field(read_fits('vAPP_phase_tutorial.fits.gz').ravel(),pupil_grid)
telescope_pupil = Field(read_fits('vAPP_amplitude_tutorial.fits.gz').ravel(),pupil_grid)

fast_axis_orientation = vAPP_phase/2


N=16
plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
imshow_field(fast_axis_orientation,mask = telescope_pupil,cmap = 'RdBu')
quiveropts = dict(color='black', headlength=0, pivot='middle', scale=30, units='xy', width=.003, headwidth=0)
X = pupil_grid.x.reshape(pupil_grid.shape)[::N,::N]

Y = pupil_grid.y.reshape(pupil_grid.shape)[::N,::N]

U = np.cos(fast_axis_orientation.shaped[::N,::N])

V = np.sin(fast_axis_orientation.shaped[::N,::N])
plt.subplot(1,2,2)
imshow_field(telescope_pupil,cmap = 'gray')
plt.quiver(X,Y,U,V,**quiveropts )
plt.gca().set_aspect('equal')
plt.show()






In [ ]:
fast_axis_orientation = (vAPP_phase - pupil_grid.y*20*2*np.pi)/2 -np.pi/2
fast_axis_orientation = fast_axis_orientation%(np.pi)#-np.pi/2
N=16
plt.figure(figsize = (10,10))
plt.subplot(1,2,1)
imshow_field(fast_axis_orientation,mask = telescope_pupil,cmap = 'RdBu')
quiveropts = dict(color='black', headlength=0, pivot='middle', scale=30, units='xy', width=.003, headwidth=0)
X = pupil_grid.x.reshape(pupil_grid.shape)[::N,::N]

Y = pupil_grid.y.reshape(pupil_grid.shape)[::N,::N]

U = np.cos(fast_axis_orientation.shaped[::N,::N])

V = np.sin(fast_axis_orientation.shaped[::N,::N])
plt.subplot(1,2,2)
imshow_field(telescope_pupil,cmap = 'gray')
plt.quiver(X,Y,U,V,**quiveropts )
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
fast_axis_orientation = vAPP_phase/2
phase_retardation = np.pi

circularity = 0

vAPP_pol = PhaseRetarder(phase_retardation,fast_axis_orientation,circularity)

wavefront = Wavefront(telescope_pupil,1 ,stokes_vector=(1,0,0,-1))

focal_grid = make_focal_grid(8, 40)

prop = FraunhoferPropagator(pupil_grid, focal_grid)


wavefront_out = vAPP_pol.forward(wavefront)


vAPP_PSF = (prop(wavefront_out).intensity)

imshow_field(np.log10(vAPP_PSF/vAPP_PSF.max()),vmin = -5)
plt.show()

In [ ]:
wavefront = Wavefront(telescope_pupil,1 ,stokes_vector=(1,0,0,1))

wavefront_out = vAPP_pol.forward(wavefront)


vAPP_PSF = (prop(wavefront_out).I)#.sum(axis = 0)

imshow_field(np.log10(vAPP_PSF/vAPP_PSF.max()),vmin = -5)
plt.show()




In [ ]:

wavefront = Wavefront(telescope_pupil,1 ,stokes_vector=(1,0,0,0))

wavefront_out = vAPP_pol.forward(wavefront)


vAPP_PSF = (prop(wavefront_out).I)#.sum(axis = 0)

imshow_field(np.log10(vAPP_PSF/vAPP_PSF.max()),vmin = -5)
plt.show()



In [ ]:
retardation = np.linspace(0,np.pi,10,endpoint=True)
wavefront = Wavefront(telescope_pupil,1 ,stokes_vector=(1,0,0,0))

fig = plt.figure(figsize= (8,8))
im = imshow_field(focal_grid.x,focal_grid, vmin=-5, vmax=0)
title = plt.title('',fontsize = 'xx-large')
plt.close()



def animate(i):
    
    vAPP_pol = PhaseRetarder(retardation[i],fast_axis_orientation,circularity)

    wavefront_out = vAPP_pol.forward(wavefront)

    vAPP_PSF = Field((prop(wavefront_out).I),focal_grid)#.sum(axis = 0)
    vAPP_PSF/= vAPP_PSF.max()
    x,y = focal_grid.coords.separated_coords
    im.set_data(x, y, np.log10(vAPP_PSF.shaped))# imshow_field(np.log10(vAPP_PSF/vAPP_PSF.max()),vmin = -5)
    title.set_text('Retardance = {0} degrees'.format(round(np.degrees(retardation[i]),0)))
    return (im,title)


ani = animation.FuncAnimation(fig, animate, range(10), blit=True)
HTML(ani.to_jshtml(default_mode = 'reflect'))



